In [ ]:
import numpy as np
import nrrd
import tifffile as tiff
from pathlib import Path
import flammkuchen as fl
import os.path
import json
import bg_space as bg

In [ ]:
master =  Path(r"\\Funes\Shared\experiments\E0020_random_coherences\v35_two_options_forward_ol\h2b")

In [ ]:
# find suite2p files: 
files_path = list(master.glob("*_f*/data_from_suite2p_cells.h5"))
files_path

In [ ]:
# For a single fish
suite2p_file = fl.load(files_path[0])
anatomy = suite2p_file['anatomy_stack']

anatomy_reorient = np.zeros_like(anatomy)
source_origin = ('Inferior', 'Right', 'Posterior')
target_origin = ('Inferior', 'Anterior', 'Right') # this is the orientation of the mpin reference 
anatomy_reorient = bg.map_stack_to(source_origin, target_origin, anatomy)

dir_path = os.path.dirname(str(files_path[0]))
file_name = Path(dir_path) / 'anatomy_suite2p_res_rot.nrrd'
file_name

In [ ]:
with open(next(Path(dir_path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
z_res = z_tot_span / n_planes

header = {'kinds': ['domain', 'domain', 'domain'], 'units': ['microns', 'microns', 'microns'], 'spacings': [z_res, 0.6, 0.6]}
nrrd.write(str(file_name), anatomy_reorient, header, index_order='C')

In [ ]:
# For all fish in master 
for f in files_path:
    print(f)
    
    suite2p_file = fl.load(f)
    anatomy = suite2p_file['anatomy_stack']
    dir_path = os.path.dirname(str(f))
    
    anatomy_reorient = np.zeros_like(anatomy)
    source_origin = ('Inferior', 'Right', 'Posterior')
    target_origin = ('Inferior', 'Anterior', 'Right') # this is the orientation of the mpin reference 
    anatomy_reorient = bg.map_stack_to(source_origin, target_origin, anatomy)
    
    file_name = Path(dir_path) / 'registration' / 'anatomy_suite2p_res_rot.nrrd'
    with open(next(Path(dir_path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
    lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
    z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
    n_planes = lsconfig["triggering"]["n_planes"]
    z_res = z_tot_span / n_planes
    
    header = {'kinds': ['domain', 'domain', 'domain'], 'units': ['microns', 'microns', 'microns'], 'spacings': [0.6, 0.6, z_res]}
    nrrd.write(str(file_name), anatomy_reorient, header, index_order='C')